In [4]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from functions  import intensity_data
from scipy.io import loadmat
sub=['F2']
# sub = ['M2']
fold=['Fold1','Fold2']
# fold = ['Fold4']
seg = 'Lower'
per = [99]
# per = [97]
for p in per:
    for s in sub:
        mat = loadmat(seg+'/'+s+'/'+"pxl_np_"+str(p)+".mat")
        np_fold = mat['pxl_np']
        for f in fold: 
            print('perc'+str(p)+'Subject'+s+'and'+f)
            #dataset = pandas.read_csv(seg+'/'+s+'/'+f+'/'+"Grid_Wise_Set_train_"+str(p)+".csv", delimiter=",",header = None)
            #dataset.replace(numpy.inf, numpy.nan)
            #dataset[dataset==numpy.inf] = numpy.nan
            #dataset.fillna(dataset.mean(), inplace=True)
            #dataset = dataset.as_matrix(columns = None)
            ind = fold.index(f)
            np = np_fold[0,ind][1][0][0]
            dataset = loadmat(seg+'/'+s+'/'+f+'/'+"Intensity_Set_"+str(p)+".mat")
            a1= dataset['intensity_profile_frame']
            l = a1.shape[1]
            l1 = int(l/4)
            a = numpy.array(a1[0][0:l-l1])
            a = a.reshape(1,l-l1)
            b = numpy.array(a1[0][l-l1:l])
            b = b.reshape(1,l1)
            a = intensity_data(a)
            b = intensity_data(b)
            print(len(a))
            print(len(b))
            X = a[:,0:np+1]
            Xorg = X.copy()
            X = X[~numpy.isnan(X).any(axis=1)]
            Y = a[:,np+1]
            Y = Y[~numpy.isnan(Xorg).any(axis=1)]
            X1 = b[:,0:np+1]
            X1org = X1.copy()
            X1 = X1[~numpy.isnan(X1).any(axis=1)]
            Y1 = b[:,np+1]
            Y1 = Y1[~numpy.isnan(X1org).any(axis=1)]
            if len(a)<=200000:
                n1 = 256
                n2 = 128
                n3 = 128
                bs = 100
            elif len(a)<=400000:
                n1 = 256
                n2 = 256
                n3 = 256
                bs = 200
            elif len(a)<=600000:
                n1 = 512
                n2 = 512
                n3 = 256
                bs = 300
            elif len(a)<=800000:
                n1 = 1024
                n2 = 512
                n3 = 512
                bs = 400
            else:
                n1 = 1024
                n2 = 1024
                n3 = 1024
                bs = 600
            def base_model():
                model = Sequential()
                model.add(Dense(n1,activation="relu", kernel_initializer="normal", input_dim=np+1))
                model.add(Dropout(0.15))
#                 if l > 20000:
                model.add(Dense(n2, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))
#                 if l > 50000:
                model.add(Dense(n3, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))  
                model.add(Dense(1, activation="relu", kernel_initializer="normal"))
                model.compile(loss='mean_squared_error',optimizer='adam')
                return model
            fname=(seg+'/'+s+'/'+f+'/'+'Model_GridWise_dev_' + str(p) + '.hdf5')
            callbacks = [EarlyStopping(monitor='val_loss', patience=4, min_delta = 0.00001), ModelCheckpoint(fname, monitor='val_loss', verbose=0, save_best_only=True, mode='auto', period=1)]
            estimator = KerasRegressor( build_fn = base_model)
            estimator.fit(X, Y, validation_data=(X1,Y1), epochs=100, batch_size=bs, callbacks=callbacks)
            a = []
            b = []
            X1 = []
            Y1 = []
            X1org = []
            Xorg = []
            X = []
            Y = []
            a1 = []
            dataset = []            
print("TRAINING COMPLETE!!")

perc99SubjectF2andFold1
526499
171533
Train on 454292 samples, validate on 148711 samples
Epoch 1/100
454292/454292 [==============================] - 86s 189us/step - loss: 0.0375 - val_loss: 0.0365
Epoch 2/100
454292/454292 [==============================] - 82s 179us/step - loss: 0.0327 - val_loss: 0.0385
Epoch 3/100
454292/454292 [==============================] - 84s 186us/step - loss: 0.0318 - val_loss: 0.0355
Epoch 4/100
454292/454292 [==============================] - 96s 211us/step - loss: 0.0312 - val_loss: 0.0359
Epoch 5/100
454292/454292 [==============================] - 94s 207us/step - loss: 0.0307 - val_loss: 0.0364
Epoch 6/100
454292/454292 [==============================] - 94s 206us/step - loss: 0.0303 - val_loss: 0.0365
Epoch 7/100
454292/454292 [==============================] - 95s 209us/step - loss: 0.0300 - val_loss: 0.0345
Epoch 8/100
454292/454292 [==============================] - 95s 210us/step - loss: 0.0297 - val_loss: 0.0346
Epoch 9/100
454292/454292 [===